In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, AvgPool2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile

zip_ref = zipfile.ZipFile("/content/drive/My Drive/Machine_Learning_SWE_427/ML_Lab_Final_Assignment/dataset.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [4]:
import cv2
import glob

abdomen = []
chest = []
cxr = []
hand = []
head = []

for i in glob.glob("/tmp/CNN_dataset/AbdomenCT/*.jpeg"):
    abdomen.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/ChestCT/*.jpeg"):
    chest.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/CXR/*.jpeg"):
    cxr.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/Hand/*.jpeg"):
    hand.append(cv2.imread(i))

for i in glob.glob("/tmp/CNN_dataset/HeadCT/*.jpeg"):
    head.append(cv2.imread(i))


In [5]:
x,y = [],[]
dataset_size = len(hand)
for i in range(dataset_size):
  x.append(abdomen[i])
  y.append(1)

  x.append(chest[i])
  y.append(2)

  x.append(cxr[i])
  y.append(3)

  x.append(hand[i])
  y.append(4)

  x.append(head[i])
  y.append(5)


In [6]:
hand[0][0][0]

array([141, 141, 141], dtype=uint8)

In [7]:
x,y = np.array(x), np.array(y)

In [8]:
#defining training and test sets
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3)

#Dimension of the dataset
print((x_train.shape,y_train.shape))
print((x_test.shape,y_test.shape))

((35000, 64, 64, 3), (35000,))
((15000, 64, 64, 3), (15000,))


In [9]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [10]:
model = tf.keras.Sequential()
# Must define the input shape in the first layer of the neural network
model.add(Conv2D(filters=64, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu', input_shape=(64,64,3))) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(filters=32, kernel_size=(2,2),strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 64)        832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        8224      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 32)        0

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
x_train = x_train.reshape(-1, 64, 64,3) #Reshape for CNN 
x_test = x_test.reshape(-1, 64, 64, 3)
x_train.shape

(35000, 64, 64, 3)

In [13]:
model_log=model.fit(x_train, y_train,
          batch_size=60,
          epochs=10,
          verbose=1,
          validation_split=.3)

Epoch 1/10
409/409 [==============================] - 19s 18ms/step - loss: 0.1293 - accuracy: 0.9572 - val_loss: 0.2018 - val_accuracy: 0.8728
Epoch 2/10
409/409 [==============================] - 7s 17ms/step - loss: 0.0187 - accuracy: 0.9946 - val_loss: 0.0189 - val_accuracy: 0.9970
Epoch 3/10
409/409 [==============================] - 6s 15ms/step - loss: 0.0095 - accuracy: 0.9972 - val_loss: 0.0282 - val_accuracy: 0.9950
Epoch 4/10
409/409 [==============================] - 7s 17ms/step - loss: 0.0077 - accuracy: 0.9977 - val_loss: 0.0051 - val_accuracy: 0.9986
Epoch 5/10
409/409 [==============================] - 6s 15ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 0.0066 - val_accuracy: 0.9984
Epoch 6/10
409/409 [==============================] - 7s 17ms/step - loss: 0.0099 - accuracy: 0.9969 - val_loss: 0.0044 - val_accuracy: 0.9986
Epoch 7/10
409/409 [==============================] - 6s 14ms/step - loss: 0.0070 - accuracy: 0.9981 - val_loss: 0.0043 - val_accuracy: 0.998